In [1]:
from helpers.log_config import setup_logging
setup_logging()

import numpy as np
import pandas as pd
from swmm_api.output_file.extract import SwmmOutExtractWarning
import warnings
import matplotlib.pyplot as plt
from src.swmmRouting.routing import Router
from src.swmmRouting.seeding import Seeder

warnings.filterwarnings("ignore", category=SwmmOutExtractWarning)

# Input Data
## Input files
please insert model- and output-paths here

In [15]:
# model input and output files
model_path = r"..\sample_data\sample_model.inp"
out_path = r"..\sample_data\sample_model.out"
# population data
pop_path = r"../sample_data/pop_data.csv"
df_population = pd.read_csv(pop_path)

## Input patterns
define the defecation pattern / how many flushes containing wipes per hour are usually done
and define how many wipes are usually flushed per event

In [3]:
# pattern for distribution of defecation events, total sum of 1.5 defecations per day
df_defpat = pd.Series(1.5 * np.array([1.4, 0.3, 0.1, 0.0, 0.3, 1.7, 9.1, 21, 13, 9, 6.9, 4.9,
                                     1.9, 3.6, 2.5, 2, 2.9, 2.3, 4.1, 4.0, 2.7, 2.1, 2.2, 2.0])/100)
# number of wipes sent per flush
wipes_per_flush = 5

## Snagging function
define the function that determines with what probability a packet snags depending on the velocity in the pipe

In [4]:
probability_by_velocity = lambda v: 0.1

## Blocking function
Define the blockage / opening of conduit depending on pile size

create a function that returns the target setting of a conduit depending on the number of snagged wipes 

In [5]:
blockage_from_pile = lambda pile: (pile / (2+pile))

## Simulation settings
define the node of interest at which snagging occurs
define the decay rate of the pile
define how often the simulation should be updated with the next pile size and blockage setting

In [24]:
target_node = "MH3329503824",
decay_rate = lambda flow, pile, hour: 0.1
pile_decay_rate = decay_rate,  # 1/hr
update_freq = "10H"

# Runner
This cell runs the model, interrupts in defined intervals, reads output data and adapts the model parameters

In [8]:
# combines all information into a dict to pass as an argument
simulation_dict = dict(
    model_path=model_path,
    out_path=out_path,
    pop_path=pop_path,
    defpat=df_defpat,
    wipes_per_flush=wipes_per_flush,
    probability_by_velocity=probability_by_velocity,
    blockage_from_pile=blockage_from_pile,
    target_node = "MH3295504178",
    pile_decay_rate = 0.1,  # 1/hr
    update_freq = "10H"
)

## prepare seeder

In [13]:
# run blockage simulation
seeder = Seeder(df_seeding_population=df_population, hourly_probability=df_defpat)

seed_table = seeder.generate_seeds(start=pd.to_datetime("01.04.2015 00:00"),
                                   end=pd.to_datetime("01.06.2015 00:00"))

## prepare router

In [18]:
router = Router()
router.get_network_from_inpfile(model_path)
router.get_flows_from_outfile(path_out=out_path,
                              start=pd.to_datetime("01.04.2015 00:00"),
                              end=pd.to_datetime("01.06.2015 00:00"))

## route packages

In [46]:
router.generate_empty_routingtable(target=target_node)


# target_node in router.g_network.nodes
# target_node = "MH3329503824"
# target_node in router.g_network.nodes

,JP3314102514,JP3335503420,JP3345803001,MH3308002676,MH3313502815,MH3313903246,MH3314603043,MH3316903340,MH3320502421,MH3323503756,...,MH3331103095,MH3333803185,MH3336503275,MH3339203364,MH3341803453,MH3344503535,MH3347403622,MH3339903679,MH3341403729,MH3329503824


# Evaluation

In [10]:
from swmm_api import read_out_file
from swmm_api.output_file import VARIABLES as swmm_vars
from swmm_api.output_file import OBJECTS as swmm_objs

## get pile size over time

In [104]:
out = read_out_file(out_path)
df_flows = out.get_part(kind=swmm_objs.LINK, variable=swmm_vars.LINK.FLOW)
df_velocities = out.get_part(kind=swmm_objs.LINK, variable=swmm_vars.LINK.VELOCITY)

In [105]:
fig, ax = plt.subplots()
df_flows["MH3295504178.1"].plot(label="flow rate", ax=ax)
ax.set_ylabel("flow rate [L/s]")
# ax2 = ax.twinx()
# df_velocities["MH3295504178.1"].plot(label="velocities", ax=ax2)
# ax2.set_ylabel("velocities [m/s]")

In [103]:
fig, axs = plt.subplots(ncols=2)
df_flows.loc["2015-04-02 00:00":"2015-04-02 00:05","MH3295504178.1"].plot(marker="o", ax=axs[0])
times = pd.to_datetime(["2015-04-02 00:00:10", "2015-04-02 00:01:30", "2015-04-02 00:02:50", "2015-04-02 00:03:50"])
flows = np.interp(times.astype(np.int64).values/1e9, df_flows["MH3295504178.1"].index.astype(np.int64)/1e9, df_flows["MH3295504178.1"].values)
df_interp = pd.DataFrame(flows, index=times)
df_interp.plot(ax=axs[1], color="tab:red", marker="x", markersize=20, zorder=20
# plt.scatter(times, 9.07*np.ones([4]), color="tab:red")

In [85]:
orig = df_flows.loc[["2015-04-02 00:00","2015-04-02 00:01"],"MH3295504178.1"]
xp = df_flows.loc[["2015-04-02 00:00","2015-04-02 00:01"],"MH3295504178.1"].index
fp = df_flows.loc[["2015-04-02 00:00","2015-04-02 00:01"],"MH3295504178.1"].values
x = pd.to_datetime("2015-04-02 00:00:30")
print(fp[0], np.interp(x.value / 1e9, xp.astype(np.int64)/1e9, fp), fp[1])

In [76]:
[pd.to_datetime("2015-04-02 00:00:30")]

In [55]:
columns = ["log_time", "logger", "level", "sim_time", "variable", "value"]
df_log = pd.read_csv(r"../helpers/application.log", sep=";", skiprows=15)
df_log.columns = columns
# df_log["log_time"] = pd.to_datetime(df_log["log_time"])
df_pile_size = df_log.loc[df_log["variable"]=="pile size", ["sim_time", "value"]]
df_pile_size["sim_time"] = pd.to_datetime(df_pile_size["sim_time"])

fig, ax = plt.subplots()
df_pile_size.set_index("sim_time").plot(ax=ax)
ax.set_xlim([df_flows.index[0], df_flows.index[-1]])

In [42]:
df_flows.index[-1]